#  DM-51659 : Generate WFS LSSTCam  data for donut correlator 

    
Use opSim to provide visit-specific information,  sourced with GAIA DR2.  


We run imSim with an extended stack version. The specific setup included 

    source /cvmfs/sw.lsst.eu/almalinux-x86_64/lsst_distrib/w_2025_29/loadLSST-ext.bash 
    setup lsst_distrib
    export IMSIM_HOME=/sdf/home/s/scichris/link_to_scichris/WORK/imsim_home
    export RUBIN_SIM_DATA_DIR=$IMSIM_HOME/rubin_sim_data
    
    export SIMS_SED_LIBRARY_DIR=$IMSIM_HOME/rubin_sim_data/sims_sed_library

    setup -k -r $IMSIM_HOME/imSim 
    setup -k -r $IMSIM_HOME/skyCatalogs
    
    
 
*versions* :  

* **lsst_distrib w_2025_29** ext 
* **imsim** : 2.0  `main` branch  commit 7046acbd7  (Jul 16, 2025)
* **skyCatalogs**  `main` branch commit 3535be  (Jul 17, 2025) 
* **galsim** 2.7.2 
* 



We generate the focus sweep by submitting the same imsim yaml config file, but changing


    input.telescope.focusZ: 0.0015   # telescope offset is in meters

    output.header:
        focusZ: 1.5   # header is in mm
        seqnum: 942

The source brightness can be limit by setting:

    input.sky_catalog:
    max_flux: 1e7

We can generate OPD since it's very quick:

In [2]:
from lsst.afw.cameraGeom import PIXELS, FIELD_ANGLE, FOCAL_PLANE
from lsst.geom import Point2D
from astropy.table import Table
import lsst.obs.lsst as obs_lsst
import numpy as np

camera =  obs_lsst.LsstCam().getCamera()

# only for instName = "lsst"   
fieldX, fieldY = list(), list()
fieldXrad, fieldYrad = list(), list()
detName = list()

all_detectors = list(camera.getNameMap().keys())

# Select the desired detectors for OPD locations: 
raft_detectors = [name  for name in all_detectors if 'SW' in name]
print(raft_detectors)

xps, yps = [],[]
for name in raft_detectors:
    detector = camera.get(name) 
    # xp, yp = detector.getCenter(FOCAL_PLANE) # in mm 
    xp_rad, yp_rad = detector.getCenter(FIELD_ANGLE)  # in radians 
    xp_deg =  np.rad2deg(xp_rad)
    yp_deg = np.rad2deg(yp_rad)
    print(name, detector.getId(), xp_deg, yp_deg)
    xps.append(xp_deg)
    yps.append(yp_deg)
    
    
# print in a format expected by imsim 

i=0
for name in raft_detectors:
    print("    - thx:", xps[i], "deg")
    print("      thy:", yps[i], "deg ")#"# ", name)
    i+=1 
    
    
    


['R44_SW0', 'R40_SW1', 'R04_SW1', 'R04_SW0', 'R00_SW0', 'R00_SW1', 'R44_SW1', 'R40_SW0']
R44_SW0 203 1.189695602847824 1.125140844711594
R40_SW1 200 -1.2542537228148514 1.1897045073834476
R04_SW1 196 1.2542537228148514 -1.1897045073834476
R04_SW0 195 1.125140844711594 -1.1896956028478243
R00_SW0 191 -1.189695602847824 -1.125140844711594
R00_SW1 192 -1.1897045073834476 -1.2542537228148514
R44_SW1 204 1.1897045073834476 1.2542537228148514
R40_SW0 199 -1.125140844711594 1.189695602847824
    - thx: 1.189695602847824 deg
      thy: 1.125140844711594 deg 
    - thx: -1.2542537228148514 deg
      thy: 1.1897045073834476 deg 
    - thx: 1.2542537228148514 deg
      thy: -1.1897045073834476 deg 
    - thx: 1.125140844711594 deg
      thy: -1.1896956028478243 deg 
    - thx: -1.189695602847824 deg
      thy: -1.125140844711594 deg 
    - thx: -1.1897045073834476 deg
      thy: -1.2542537228148514 deg 
    - thx: 1.1897045073834476 deg
      thy: 1.2542537228148514 deg 
    - thx: -1.12514084471

In [ ]:
191,192, 195,196,  199,200,  203,204

We create a simulation directory at USDF: 

    mkdir /sdf/group/rubin/shared/scichris/DM-51659_correlator_WFS_simulation

    

We put all the information into `imsim-run-WFS-defocal.yaml` : 

In [3]:
def print_imsim_yaml(file_name = 'imsim-run-WFS-defocal.yaml', 
                     out_dir = '/sdf/group/rubin/shared/scichris/DM-51659_correlator_WFS_simulation',
                    ):
    content = '''
modules: [imsim]
template: imsim-config-skycat

# Use skyCatalogs for obtaining the objects to render.
input.sky_catalog:
  file_name: /sdf/data/rubin/user/jchiang/imSim/skyCatalogs/skyCatalog.yaml
  approx_nobjects: 1000
  band: { type: OpsimData, field: band }
  mjd: { type: OpsimData, field: mjd }
  obj_types: [gaia_star]
  max_flux: 1e7 

input.opsim_data.file_name: /sdf/data/rubin/user/jchiang/imSim/rubin_sim_data/opsim_cadences/baseline_v3.2_10yrs.db
input.opsim_data.visit: 740000

input.atm_psf.screen_size: 819.2
input.atm_psf.save_file:
  type: FormattedStr
  format: atm_psf_files/atm_psf_%08d-%1d-%s.pkl
  items:
      - { type: OpsimData, field: observationId }
      - { type: OpsimData, field: snap }
      - { type: OpsimData, field: band }

# offset the piston by 1.5 mm 
input.telescope.focusZ: 0   # telescope offset is in meters

# disable checkpointing
input.checkpoint: ""

image.random_seed: '@input.opsim_data.visit'

# simulate no objects for OPD-only
#image.nobjects: 0

output.nproc: 8
output.det_num: [191, 192, 195, 196, 199, 200, 203, 204]
output.nfiles: 8

# make no amp images
#output.readout: ""

output.header:
    focusZ: 0.0   # header is in mm 
    seqnum: 941

output.camera: LsstCam
output.dir:
    type: FormattedStr
    format : output/%08d
    items:
        - "@input.opsim_data.visit"

# apply specific formatting to the amp images 
output.readout.file_name:
    type: FormattedStr
    format : amp_%08d-%1d-%s-%s-det%03d-%s.fits.fz
    items:
        - "@input.opsim_data.visit"  # eg. 00740000
        - 0   # snap 
        - $band  # eg. g
        - $det_name  # eg. R22_S10 
        - "@output.det_num"  # eg. 91--> 091  
        - "@input.telescope.focusZ" #  eg. 0.0015 

output.timeout: 1e5
output.truth.dir: '@output.dir'
output.truth.file_name.format: centroid_%08d-%1d-%s-%s-det%03d.txt.gz
'''
    print(content)

We simulate different `focusZ`  ensuring that each image  gets a different `seqNum`, since that's used to create the exposure number, and allows for butler ingestion, eg.  


    galsim imsim-run-WFS-defocal.yaml   input.telescope.focusZ=-0.0015 output.header.focusZ=-1.5    output.header.seqnum=940  input.opsim_data.visit=68659  

Generate slurm submission scripts. We change the visit number,  focusZ in the `input.telescope` and `output.header`, as well as `seqNum` in the header:

In [23]:
import os 
nodes=4
thrs= 100 # for some jobs 5 hrs this is insufficient
partition='milano'

#mem=100 # GB, should be generally output.nproc*6GB , so here 9*6GB = 54 GB 
mem = 300
visit = 68659
seqNum = 940

def write_to_file(out_file, content):
    with open(out_file, "w") as output:
        for line in content:
            output.write(line)
# Note that in the output directory we need to make the `atm_psf_files` 
# directory (it doesn't get made automatically, as eg `output_all_R22` with the amp files does):
path_cwd = '/sdf/data/rubin/shared/scichris/DM-51659_correlator_WFS_simulation/'
atm_files_dir = os.path.join(path_cwd, 'atm_psf_files')
if not os.path.exists(atm_files_dir):
    os.makedirs(atm_files_dir)

for focusz_mm in  np.linspace(-0.0015, 0, 10):
    slurm_file = os.path.join(path_cwd, f'runSlurm_lsstcam-{visit}-{seqNum}_noFFT.sl')

    # the instance catalog to use ... 
    path_to_imsim_yaml = os.path.join(path_cwd,'imsim-run-WFS-defocal.yaml')
    cmd = f"galsim {path_to_imsim_yaml} input.opsim_data.visit={visit}\
    input.telescope.focusZ={focusz_mm} output.header.focusZ={1000*focusz_mm}\
    output.header.seqnum={seqNum}"

    path_to_slurm_log = os.path.join(path_cwd, f'slurm_out/{visit}_{seqNum}.out')
    content = ['#!/bin/bash -l \n',
              f'#SBATCH --partition {partition} \n',
              '#SBATCH --account rubin:developers \n',
              f'#SBATCH --nodes {nodes} \n',
              f'#SBATCH --mem={mem}G \n',
              f'#SBATCH --cpus-per-task=8\n',
              f'#SBATCH -t {thrs}:00:00 \n', 
              f'#SBATCH --job-name {visit}{seqNum} \n'
              f'#SBATCH --output={path_to_slurm_log} \n',
                'echo "starting at `date` on `hostname`" \n',
                "pwd \n",
                 cmd,
                '\n echo "ended at `date` on `hostname`" \n',
              ]
    write_to_file(slurm_file, content)
    print(visit, seqNum,  focusz_mm, slurm_file)
    seqNum += 1 # ensure it's different for each defocal offset


68659 940 -0.0015 /sdf/data/rubin/shared/scichris/DM-51659_correlator_WFS_simulation/runSlurm_lsstcam-68659-940_noFFT.sl
68659 941 -0.0013333333333333333 /sdf/data/rubin/shared/scichris/DM-51659_correlator_WFS_simulation/runSlurm_lsstcam-68659-941_noFFT.sl
68659 942 -0.0011666666666666668 /sdf/data/rubin/shared/scichris/DM-51659_correlator_WFS_simulation/runSlurm_lsstcam-68659-942_noFFT.sl
68659 943 -0.001 /sdf/data/rubin/shared/scichris/DM-51659_correlator_WFS_simulation/runSlurm_lsstcam-68659-943_noFFT.sl
68659 944 -0.0008333333333333334 /sdf/data/rubin/shared/scichris/DM-51659_correlator_WFS_simulation/runSlurm_lsstcam-68659-944_noFFT.sl
68659 945 -0.0006666666666666668 /sdf/data/rubin/shared/scichris/DM-51659_correlator_WFS_simulation/runSlurm_lsstcam-68659-945_noFFT.sl
68659 946 -0.0005 /sdf/data/rubin/shared/scichris/DM-51659_correlator_WFS_simulation/runSlurm_lsstcam-68659-946_noFFT.sl
68659 947 -0.0003333333333333335 /sdf/data/rubin/shared/scichris/DM-51659_correlator_WFS_simul

We can submit all these with `submit_slurm.py` script:
    

In [19]:
def make_submit_slurm(path_cwd):
    content = '''
# contents of `submit_slurm.py` script
# need to be run from terminal where
# imsim is available 
import subprocess
import os
flist = [f  for f in os.listdir(os.getcwd()) if f.startswith('runSlurm') ]
for i in range(len(flist)):
    slurm_file = flist[i]
    print(f"Running sbatch  {slurm_file}")
    subprocess.call(["sbatch", slurm_file])
'''
    out_file = os.path.join(path_cwd, 'submit_slurm.py')
    write_to_file(out_file, content) 
    print('Saved ', out_file)
    
make_submit_slurm(path_cwd)

Saved  /sdf/data/rubin/shared/scichris/DM-51659_correlator_WFS_simulation/submit_slurm.py


We can check with `squeue -u scichris ` that they are running. We can show here that the files get generated, and check progress in the log files eg. `68659_940.outnoFFT`  

We generate a gen3 repo:

In [24]:
from lsst.ts.wep.utils import  runProgram
butlerRootPath = os.path.join(path_cwd, 'gen3repo')
runProgram(f"butler create {butlerRootPath}")
runProgram(f"butler register-instrument {butlerRootPath} lsst.obs.lsst.LsstCam")

We ingest the GAIA DR2 refcat: 


In [ ]:
ecsvPath = "/sdf/data/rubin/repo/aos_imsim/gaia_dr2_20200414.ecsv"
collection = "refcats/gaia_dr2_20200414"
datasetType = "gaia_dr2_20200414"
path_cwd = '/sdf/data/rubin/shared/scichris/DM-41679/'
butlerRootPath = os.path.join(path_cwd, 'gen3repo_noFFT')

runProgram(f"butler register-dataset-type {butlerRootPath}"
           f" {datasetType} SimpleCatalog htm7")
runProgram(f"butler ingest-files -t direct {butlerRootPath}"
           f"  {datasetType} {collection}  {ecsvPath} --prefix /sdf/group/rubin")
runProgram(f"butler collection-chain {butlerRootPath} --mode extend refcats {collection}")

Write calibs once:

In [ ]:
from lsst.daf import butler as dafButler
from lsst.ts.wep.utils import  runProgram

butler = dafButler.Butler(butlerRootPath)
butlerInstName = 'Cam'
if f"LSST{butlerInstName}/calib" not in butler.registry.queryCollections():
    print("Ingesting curated calibrations.")

    runProgram(
        f"butler write-curated-calibrations {butlerRootPath} lsst.obs.lsst.Lsst{butlerInstName}"
    )

Ingest raws:

In [ ]:
outputImgDir = os.path.join(path_cwd, 'output', '00068659')

In [ ]:
runProgram(f"butler ingest-raws {butlerRootPath} {outputImgDir}/amp*")
runProgram(f"butler define-visits {butlerRootPath} lsst.obs.lsst.LsstCam")
    

See what got ingested:

In [ ]:
butler = dafButler.Butler(butlerRootPath)
registry = butler.registry
datasetRefs = butler.registry.queryDatasets(datasetType='raw',collections=['LSSTCam/raw/all']).expanded()
for ref  in datasetRefs:
    print(ref)

Run ISR to get the postISRCCD: 

First run ISR only, since WEP wouldn't know what to do with an in-focus image:

In [ ]:
from lsst.ts.wep.utils import getConfigDir as getWepConfigDir
def writeWepConfigurationIsrOnly(instName, pipelineYamlPath):
        """Write wavefront estimation pipeline task configuration.

        Parameters
        ----------
        instName : str
            Name of the instrument this configuration is intended for.
        pipelineYamlPath : str
            Path where the pipeline task configuration yaml file
            should be saved.
        filterTypeName : str
            Filter type name: ref (or ''), u, g, r, i, z, or y.
        """

        butlerInstName = "Cam"

        with open(pipelineYamlPath, "w") as fp:
            fp.write(
                f"""# This yaml file is used to define the tasks and configuration of
# a Gen 3 pipeline used for testing
description: basic processing pipeline with imsim
# Here we specify the corresponding instrument for the data we
# will be using.
instrument: lsst.obs.lsst.Lsst{butlerInstName}
# Use imported instrument configuration
#imports:
#  - location: {getWepConfigDir()}/cwfs/instData/{instName}/instParamPipeConfig.yaml
# Then we can specify each task in our pipeline by a name
# and then specify the class name corresponding to that task
tasks:
  isr:
    class: lsst.ip.isr.isrTask.IsrTask
    # Below we specify the configuration settings we want to use
    # when running the task in this pipeline. Since our data doesn't
    # include bias or flats we only want to use doApplyGains and
    # doOverscan in our isr task.
    config:
      connections.outputExposure: 'postISRCCD'
      doBias: False
      doVariance: False
      doLinearize: False
      doCrosstalk: False
      doDefect: False
      doNanMasking: False
      doInterpolate: False
      doBrighterFatter: False
      doDark: False
      doFlat: False
      doApplyGains: True
      doFringe: False
      doOverscan: True
      python: OverscanCorrectionTask.ConfigClass.fitType = 'MEDIAN'
"""
            )
instName = 'lsst'
pipelineYamlPath = os.path.join(path_cwd, "lsstPipelineISR.yaml")

writeWepConfigurationIsrOnly(instName, pipelineYamlPath)   


Test the query : 

In [ ]:
datasetRefs = registry.queryDatasets('raw',collections=['LSSTCam/raw/all'],
                                     where=f"instrument='LSSTCam' and exposure.day_obs=20250820").expanded()

In [ ]:
for ref in datasetRefs:
    print(ref)

In [ ]:
len(list(datasetRefs))

Run ISR with that query:

In [ ]:
# 1) run ISR only : seleect by obs day since there aren;t any other visits we're simulating now ...
butlerInstName = 'Cam'
runName = 'run1'
numPro=5
instName = 'lsst'
pipelineYamlPath = os.path.join(path_cwd, "lsstPipelineISR.yaml")
runProgram(f"pipetask run -b {butlerRootPath} "
            f"-i LSST{butlerInstName}/raw/all,LSST{butlerInstName}/calib/unbounded "
            f"--instrument lsst.obs.lsst.Lsst{butlerInstName} "
            f"--register-dataset-types --output-run {runName}  -p {pipelineYamlPath} -d "
           f'"exposure.day_obs=20250820" -j {numPro}'
          )

# or use exposure.science_program={}

lsst.isr INFO: Widening saturation trails.
lsst.isr INFO: Widening saturation trails.
lsst.isr INFO: Applying gain correction instead of flat.
lsst.isr INFO: Applying gain correction instead of flat.
lsst.isr WARNING: No rough magnitude zero point defined for filter g_6.
lsst.isr INFO: Setting rough magnitude zero point for filter g_6: 30.940228
lsst.isr WARNING: No rough magnitude zero point defined for filter g_6.
lsst.isr INFO: Setting rough magnitude zero point for filter g_6: 30.940228
lsst.isr WARNING: No rough magnitude zero point defined for filter g_6.
lsst.isr INFO: Setting rough magnitude zero point for filter g_6: 30.940228
lsst.isr WARNING: No rough magnitude zero point defined for filter g_6.
lsst.isr INFO: Setting rough magnitude zero point for filter g_6: 30.940228
lsst.isr WARNING: No rough magnitude zero point defined for filter g_6.
lsst.isr INFO: Setting rough magnitude zero point for filter g_6: 30.940228
lsst.ctrl.mpexec.singleQuantumExecutor INFO: Execution of ta

lsst.isr INFO: Widening saturation trails.
lsst.isr INFO: Applying gain correction instead of flat.
lsst.isr INFO: Widening saturation trails.
lsst.isr INFO: Applying gain correction instead of flat.
lsst.ip.isr.calibType WARNING: Invalid PTC turnoff value for C10: nan Setting to default: PTC Turnoff=2e19
lsst.ip.isr.calibType WARNING: Invalid PTC turnoff value for C11: nan Setting to default: PTC Turnoff=2e19
lsst.ip.isr.calibType WARNING: Invalid PTC turnoff value for C12: nan Setting to default: PTC Turnoff=2e19
lsst.ip.isr.calibType WARNING: Invalid PTC turnoff value for C13: nan Setting to default: PTC Turnoff=2e19
lsst.ip.isr.calibType WARNING: Invalid PTC turnoff value for C14: nan Setting to default: PTC Turnoff=2e19
lsst.ip.isr.calibType WARNING: Invalid PTC turnoff value for C15: nan Setting to default: PTC Turnoff=2e19
lsst.ip.isr.calibType WARNING: Invalid PTC turnoff value for C16: nan Setting to default: PTC Turnoff=2e19
lsst.ip.isr.calibType WARNING: Invalid PTC turnoff 

lsst.isr INFO: Widening saturation trails.
lsst.isr INFO: Widening saturation trails.
lsst.isr INFO: Applying gain correction instead of flat.
lsst.isr INFO: Applying gain correction instead of flat.
lsst.ip.isr.calibType WARNING: Invalid PTC turnoff value for C10: nan Setting to default: PTC Turnoff=2e19
lsst.ip.isr.calibType WARNING: Invalid PTC turnoff value for C11: nan Setting to default: PTC Turnoff=2e19
lsst.ip.isr.calibType WARNING: Invalid PTC turnoff value for C12: nan Setting to default: PTC Turnoff=2e19
lsst.ip.isr.calibType WARNING: Invalid PTC turnoff value for C13: nan Setting to default: PTC Turnoff=2e19
lsst.ip.isr.calibType WARNING: Invalid PTC turnoff value for C14: nan Setting to default: PTC Turnoff=2e19
lsst.ip.isr.calibType WARNING: Invalid PTC turnoff value for C15: nan Setting to default: PTC Turnoff=2e19
lsst.ip.isr.calibType WARNING: Invalid PTC turnoff value for C16: nan Setting to default: PTC Turnoff=2e19
lsst.ip.isr.calibType WARNING: Invalid PTC turnoff 

lsst.isr INFO: Widening saturation trails.
lsst.isr INFO: Widening saturation trails.
lsst.isr INFO: Applying gain correction instead of flat.
lsst.isr INFO: Applying gain correction instead of flat.
lsst.ip.isr.calibType WARNING: Invalid PTC turnoff value for C10: nan Setting to default: PTC Turnoff=2e19
lsst.ip.isr.calibType WARNING: Invalid PTC turnoff value for C11: nan Setting to default: PTC Turnoff=2e19
lsst.ip.isr.calibType WARNING: Invalid PTC turnoff value for C12: nan Setting to default: PTC Turnoff=2e19
lsst.ip.isr.calibType WARNING: Invalid PTC turnoff value for C13: nan Setting to default: PTC Turnoff=2e19
lsst.ip.isr.calibType WARNING: Invalid PTC turnoff value for C14: nan Setting to default: PTC Turnoff=2e19
lsst.ip.isr.calibType WARNING: Invalid PTC turnoff value for C15: nan Setting to default: PTC Turnoff=2e19
lsst.ip.isr.calibType WARNING: Invalid PTC turnoff value for C16: nan Setting to default: PTC Turnoff=2e19
lsst.ip.isr.calibType WARNING: Invalid PTC turnoff 

lsst.isr INFO: Converting exposure to floating point values.
lsst.isr INFO: Converting exposure to floating point values.
lsst.isr INFO: Converting exposure to floating point values.
lsst.ip.isr.calibType WARNING: Invalid PTC turnoff value for C10: nan Setting to default: PTC Turnoff=2e19
lsst.ip.isr.calibType WARNING: Invalid PTC turnoff value for C11: nan Setting to default: PTC Turnoff=2e19
lsst.ip.isr.calibType WARNING: Invalid PTC turnoff value for C12: nan Setting to default: PTC Turnoff=2e19
lsst.ip.isr.calibType WARNING: Invalid PTC turnoff value for C13: nan Setting to default: PTC Turnoff=2e19
lsst.ip.isr.calibType WARNING: Invalid PTC turnoff value for C14: nan Setting to default: PTC Turnoff=2e19
lsst.ip.isr.calibType WARNING: Invalid PTC turnoff value for C15: nan Setting to default: PTC Turnoff=2e19
lsst.ip.isr.calibType WARNING: Invalid PTC turnoff value for C16: nan Setting to default: PTC Turnoff=2e19
lsst.ip.isr.calibType WARNING: Invalid PTC turnoff value for C17: na

lsst.isr WARNING: No rough magnitude zero point defined for filter g_6.
lsst.isr INFO: Setting rough magnitude zero point for filter g_6: 30.940228
lsst.isr INFO: Converting exposure to floating point values.
lsst.ip.isr.calibType WARNING: Invalid PTC turnoff value for C10: nan Setting to default: PTC Turnoff=2e19
lsst.ip.isr.calibType WARNING: Invalid PTC turnoff value for C11: nan Setting to default: PTC Turnoff=2e19
lsst.ip.isr.calibType WARNING: Invalid PTC turnoff value for C12: nan Setting to default: PTC Turnoff=2e19
lsst.ip.isr.calibType WARNING: Invalid PTC turnoff value for C13: nan Setting to default: PTC Turnoff=2e19
lsst.ip.isr.calibType WARNING: Invalid PTC turnoff value for C14: nan Setting to default: PTC Turnoff=2e19
lsst.ip.isr.calibType WARNING: Invalid PTC turnoff value for C15: nan Setting to default: PTC Turnoff=2e19
lsst.ip.isr.calibType WARNING: Invalid PTC turnoff value for C16: nan Setting to default: PTC Turnoff=2e19
lsst.ip.isr.calibType WARNING: Invalid PTC

Show the postISR images:

In [ ]:
from lsst.daf import butler as dafButler


butler = dafButler.Butler(butlerRootPath)

runName = 'run1'

exposure_intra = butler.get('postISRCCD', dataId={'instrument':'LSSTCam', 'detector':93,  # this one has focusZ -0.0015 
                                           'exposure':5025082000940}, collections=[runName])

exposure_focus = butler.get('postISRCCD', dataId={'instrument':'LSSTCam', 'detector':93,  # this one has focusZ 0.0015 
                                           'exposure':5025082000941}, collections=[runName])

exposure_extra  = butler.get('postISRCCD', dataId={'instrument':'LSSTCam', 'detector':93,  # this one has focusZ 0.0015 
                                           'exposure':5025082000942}, collections=[runName])

In [ ]:
meta = exposure_intra.getMetadata()

In [ ]:
#print(meta)

In [ ]:
import matplotlib.pyplot as plt
from astropy.visualization import ZScaleInterval
fig,ax = plt.subplots(1,3, figsize=(14,5))

zscale = ZScaleInterval()
i=0
for exposure in [exposure_intra, exposure_focus, exposure_extra]:
    d = exposure.image.array
    focusz = exposure.visitInfo.focusZ
    vmin,vmax = zscale.get_limits(d)
    mappable = ax[i].imshow(d, vmin=vmin, vmax=vmax, origin='lower')
    
    ax[i].set_title(f'{exposure.visitInfo.instrumentLabel} {exposure.detector.getName()}, offset {focusz} mm ', )
    i += 1 
#fig.suptitle( f'detector: {exposure.detector.getId()} ({exposure.detector.getName()})')
#plt.colorbar(mappable, ax=ax[1]) 
fig.subplots_adjust(wspace=0.29)

Below we show the result of turning off the FFT mode - removing ` max_flux: 1e7 ` setting, and adding 
   

    stamp.diffraction_fft: ""
    
    

In [ ]:
import matplotlib.pyplot as plt
from astropy.visualization import ZScaleInterval
fig,ax = plt.subplots(1,3, figsize=(14,5))

zscale = ZScaleInterval()
i=0
for exposure in [exposure_intra, exposure_focus, exposure_extra]:
    d = exposure.image.array
    focusz = exposure.visitInfo.focusZ
    vmin,vmax = zscale.get_limits(d)
    mappable = ax[i].imshow(d, vmin=vmin, vmax=vmax, origin='lower')
    
    ax[i].set_title(f'focusZ: {focusz}')
    i += 1 
#fig.suptitle( f'detector: {exposure.detector.getId()} ({exposure.detector.getName()})')
#plt.colorbar(mappable, ax=ax[1]) 
fig.subplots_adjust(wspace=0.29)

Then run WEP, feeding as an input the postISRCCD. We use https://github.com/lsst-ts/ts_wep/blob/develop/tests/testData/pipelineConfigs/testCalcZernikesScienceSensorPipeline.yaml and https://github.com/lsst-ts/ts_wep/blob/develop/tests/testData/pipelineConfigs/testDonutCatWcsPipeline.yaml as an inspiration:


In [ ]:
from lsst.ts.wep.utils import getConfigDir as getWepConfigDir
def writeWepConfigurationWepOnly(instName, pipelineYamlPath):
        """Write wavefront estimation pipeline task configuration.

        Parameters
        ----------
        instName : str
            Name of the instrument this configuration is intended for.
        pipelineYamlPath : str
            Path where the pipeline task configuration yaml file
            should be saved.
        filterTypeName : str
            Filter type name: ref (or ''), u, g, r, i, z, or y.
        """

        butlerInstName = "Cam"

        with open(pipelineYamlPath, "w") as fp:
            fp.write(
                f"""# This yaml file is used to define the tasks and configuration of
# a Gen 3 pipeline used for testing
description: basic processing pipeline with imsim
# Here we specify the corresponding instrument for the data we
# will be using.
instrument: lsst.obs.lsst.Lsst{butlerInstName}
# Use imported instrument configuration
#imports:
#  - location: {getWepConfigDir()}/cwfs/instData/{instName}/instParamPipeConfig.yaml
# Then we can specify each task in our pipeline by a name
# and then specify the class name corresponding to that task
tasks:
  generateDonutCatalogWcsTask:
    class: lsst.ts.wep.task.generateDonutCatalogWcsTask.GenerateDonutCatalogWcsTask
    config:
    # this config points to the GAIA DR2 refcat
      connections.refCatalogs: gaia_dr2_20200414
      anyFilterMapsToThis: phot_g_mean
      donutSelector.useCustomMagLimit: True
      donutSelector.magMax: 25.0
      donutSelector.magMin: 10.0
      donutSelector.unblendedSeparation: 1
  cutOutDonutsScienceSensorTask:
    class: lsst.ts.wep.task.cutOutDonutsScienceSensorTask.CutOutDonutsScienceSensorTask
    config:
      # And here we specify the configuration settings originally defined in
      # CutOutDonutsScienceSensorTaskConfig.
      # Test Science Sensor pipeline works when specifying instrument configuration
      donutTemplateSize: 160
      donutStampSize: 160
      initialCutoutPadding: 40
  calcZernikesTask:
    class: lsst.ts.wep.task.calcZernikesTask.CalcZernikesTask
    """)
    
    
instName = 'lsst'
pipelineYamlPath = os.path.join(path_cwd, "lsstPipelineWEP.yaml")
writeWepConfigurationWepOnly(instName, pipelineYamlPath)   



Note: ISR and WEP must be run with the same `lsst_distrib`. Otherwise we'll get a version mismatch ... 

In [ ]:
from lsst.ts.wep.utils import  runProgram
butlerInstName = 'Cam'
runName = 'run1'
numPro=5
instName = 'lsst'
path_cwd = '/sdf/data/rubin/shared/scichris/DM-41679/'
butlerRootPath = os.path.join(path_cwd, 'gen3repo')
pipelineYamlPath = os.path.join(path_cwd, "lsstPipelineWEP.yaml")
for detNum in range(90,99):
    cmd = f"pipetask run -b {butlerRootPath} "+\
                f"-i refcats/gaia_dr2_20200414,{runName},LSST{butlerInstName}/calib/unbounded "+\
                f"--instrument lsst.obs.lsst.Lsst{butlerInstName} "+\
                f"--register-dataset-types --output-run {runName}  --extend-run  -p {pipelineYamlPath} -d "+\
               f'"exposure.seq_num in (940,942) and detector={detNum}" -j {numPro}'
    print('Running', cmd)
    runProgram(cmd)

Show the outcome of running WEP:

In [ ]:
from lsst.daf import butler as dafButler
path_cwd = '/sdf/data/rubin/shared/scichris/DM-41679/'
butlerRootPath = os.path.join(path_cwd, 'gen3repo')
butler = dafButler.Butler(butlerRootPath)
registry = butler.registry
collectionsList = list(registry.queryCollections())
print(collectionsList)


registry = butler.registry
datasetTypes = list(registry.queryDatasetTypes())
for dt in datasetTypes:
    print(dt)
    

In [ ]:
datasetRefs = butler.registry.queryDatasets(datasetType='postISRCCD',
                                            collections=['run1']).expanded()
for ref  in datasetRefs:
    print(ref)

In [ ]:
datasetRefs = butler.registry.queryDatasets(datasetType='zernikeEstimateRaw',
                                            collections=['run1']).expanded()
for ref  in datasetRefs:
    print(ref)

In [ ]:
datasetRefs = registry.queryDatasets("zernikeEstimateRaw",collections='run1',
              where=f"instrument='LSSTCam'").expanded()

for ref in datasetRefs:
    print(ref)

Load the exposure:

In [ ]:
from lsst.daf import butler as dafButler

collection = 'run1'
instrument= 'LSSTCam'
detector=90
exposure_number = 5025082000942

# construct a dataId  for postISR extra-focal 
data_id = {
    "detector": detector,
    "instrument": instrument,
    "exposure": exposure_number,
}

# read the postISR exposure
exposure_extra = butler.get("postISRCCD", data_id, collections=[collection])

data_id = { "detector": detector, "instrument": instrument, "exposure": 5025082000941}
exposure_focus = butler.get("postISRCCD", data_id, collections=[collection])

Load the donut catalog:

In [ ]:

    
# construct a dataId for zernikes and donut catalog:
# switch exposure to visit
data_id = {"detector": detector, 
           "instrument": instrument, 
           "visit": exposure_number
          }

# the raw Zernikes are all stored for the extra-focal visit ,,., 
zernikes_raw = butler.get(
    "zernikeEstimateRaw", dataId=data_id, collections=[collection]
)


donut_stamps = butler.get(
    "donutStampsExtra", dataId=data_id, collections=[collection]
)



In [ ]:
len(donut_stamps)

In [ ]:
donut_catalog = butler.get(
    "donutCatalog",  dataId=data_id, collections=[collection]
    )


In [ ]:
metadata  = butler.get("isr_metadata",  dataId={
                       "detector": detector,
                       "instrument": instrument,
                       "exposure": exposure_number,},
                        collections=[collection]
                       )


In [ ]:
donut_catalog[donut_catalog['source_flux'] > 1e8]


In [ ]:
import numpy as np 
# from exposure
# RA = 306.66150424024
# DEC = -86.924706187358 
mean_ra_deg = np.rad2deg(np.mean(donut_catalog['coord_ra']))
mean_dec_deg = np.rad2deg(np.mean(donut_catalog['coord_dec']))

In [ ]:
mean_dec_deg

Overplot the refcat sources on top of the WEP detections. Query the refcat around the fieldRA, fieldDec, that correspond to the visit boresight:


In [ ]:
import lsst.daf.butler as daf_butler
from lsst.meas.algorithms import ReferenceObjectLoader
import lsst.geom

collection='refcats/gaia_dr2_20200414' 
dstype='gaia_dr2_20200414'

butler = daf_butler.Butler(butlerRootPath,
                           collections=['refcats/gaia_dr2_20200414'])
refs = set(butler.registry.queryDatasets(dstype))

refCats = [daf_butler.DeferredDatasetHandle(butler, _, {})
           for _ in refs]


dataIds = [butler.registry.expandDataId(_.dataId) for _ in refs]
config = ReferenceObjectLoader.ConfigClass()
config.filterMap = {f'{_}': f'phot_{_}_mean' for _ in ('g', 'bp', 'rp')}
ref_obj_loader = ReferenceObjectLoader(dataIds=dataIds,
                                       refCats=refCats,
                                       config=config)

# fieldRA, fieldDec correspond to the pointing of the entire visit,
# but for R22 that works fine since it's the center raft 

# otherwise it would be better to read the centroid file
# and take the mean of the centroid file 
ra = lsst.geom.Angle(mean_ra_deg, lsst.geom.degrees)
dec = lsst.geom.Angle(mean_dec_deg, lsst.geom.degrees)

center = lsst.geom.SpherePoint(ra, dec)
radius = lsst.geom.Angle(500, lsst.geom.arcseconds)
refcat_region = lsst.sphgeom.Circle(center.getVector(), radius)
band = 'bp'
cat = ref_obj_loader.loadRegion(refcat_region, band).refCat
df = cat.asAstropy().to_pandas().sort_values('id')



In [ ]:
len(df)

In [ ]:
len(donut_catalog['coord_ra'])#[m]#, donut_catalog['coord_dec'][m], marker='+',


In [ ]:
donutb_catalog

In [ ]:
exposure = butler.get("postISRCCD", data_id, collections=[collection])

In [ ]:
wcs = exposure.getWcs()
x,y = wcs.skyToPixelArray(df['coord_ra'], df['coord_dec'])


In [ ]:
plt.scatter(donut_catalog['coord_ra'], donut_catalog['coord_dec'])
plt.scatter(df['coord_ra'], df['coord_dec'])

In [ ]:
import matplotlib.pyplot as plt
m = donut_catalog['source_flux']>1e7
plt.scatter(x,y)

plt.scatter(donut_catalog['centroid_x'][m], donut_catalog['centroid_y'][m], 
           s=20, c='red', )

plt.imshow(exposure.image.array,origin='lower')

In [ ]:
import astropy.units as u
from astropy.coordinates import SkyCoord
c1 = SkyCoord(ra=donut_catalog['coord_ra'].values*u.rad, 
             dec=donut_catalog['coord_dec'].values*u.rad)


In [ ]:
c2 = SkyCoord(ra=df['coord_ra'].values*u.rad, 
               dec=df['coord_dec'].values*u.rad)
idx, d2d, d3d = c1.match_to_catalog_sky(c2)

In [ ]:
len(c1)

In [ ]:
len(c2)

In [ ]:
# indices into c2 corresponding to each entry from c1
len(idx) 

In [ ]:
c1[:10]

In [ ]:
c2[idx][:10]

In [ ]:
import pandas as pd
donut_catalog_ext = pd.merge(donut_catalog, df.iloc[idx], how='inner')#, axis=1,ignore_index=True)

In [ ]:
len(donut_catalog_ext)

In [ ]:
f= 1e7
mag = (f*u.nJy).to(u.ABmag)
print(mag)

In [ ]:
flux = donut_catalog_ext['phot_g_mean_flux'].values
mag = (flux*u.nJy).to(u.ABmag)
#mag = -2.5 * np.log10(1e9*flux) + 8.6


donut_catalog_ext['phot_g_mean_mag'] = mag

In [ ]:
donut_catalog_ext

In [ ]:
mags = donut_catalog_ext['phot_g_mean_mag'].values.value

In [ ]:
#mags < 13 

In [ ]:
d = exposure_extra.image.array
vmin,vmax = zscale.get_limits(d)
plt.imshow(d,origin='lower', vmin=vmin, vmax=vmax)
m = donut_catalog_ext['phot_g_mean_mag'].values.value < 13.0
plt.scatter(donut_catalog_ext['centroid_x'][m], donut_catalog_ext['centroid_y'][m],
            facecolors='none', edgecolors='r')

In [ ]:
d = exposure_focus.image.array
vmin,vmax = zscale.get_limits(d)
plt.imshow(d,origin='lower', vmin=vmin, vmax=vmax)
m = donut_catalog_ext['phot_g_mean_mag'].values.value < 15
plt.scatter(donut_catalog_ext['centroid_x'][m], donut_catalog_ext['centroid_y'][m],
            facecolors='none', edgecolors='r')

In [ ]:
np.sum(donut_catalog_ext['phot_g_mean_mag'].values.value < 13)

In [ ]:
m = donut_catalog_ext['phot_g_mean_mag'].values.value < 13

In [ ]:
donut_catalog_ext[m]

What's going on? Why are totally different places covered?  Really confused...

Look at OPD : 

In [ ]:
from astropy.io import fits

# one OPD file for all field locations 
path_to_dir = '/sdf/group/rubin/shared/scichris/DM-41679/output_all_R22/00068659'


opdPerOffset = {}

for fname, defocal in zip(['opd_0.0015.fits.fz','opd_-0.0015.fits.fz','opd_0.0.fits.fz'],
                       ['intra', 'extra', 'focal']
                         ):

    path_to_file = os.path.join(path_to_dir, fname)
    hdul = fits.open(path_to_file)

    opdPerDet = {}

    for hduIdx in range(1,10):#hduIdx = 0

        zkOpd = []
        #hduIdx = detector_id_to_hduIdx[detector] # For R22_S10  
        hduHeader = hdul[hduIdx].header
        for item in hduHeader:
            if item.startswith('AZ'):
                value = hduHeader[item]
                print(item, value, "[nm]")

                zkOpd.append(value)
        opdPerDet[hduIdx] = zkOpd
    opdPerOffset[defocal] = opdPerDet

In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
fig,ax = plt.subplots(1,1,figsize=(6,4))
# zkOpd in the header is zk1:zk28. So need to select only from zk4:22
colors = {'intra':'blue', 'extra':'red', 'focal':'green'}
for defocal in opdPerOffset.keys():
    opdPerDet = opdPerOffset[defocal]
    for hduIdx in range(1,10):
        zkOpd = opdPerDet[hduIdx]
        ax.plot(range(4,23), np.array(zkOpd[3:22])/1000., label=f'det {hduIdx}',
               c=colors[defocal])
    ax.set_title('header-based OPD, ')
    #ax.legend(bbox_to_anchor=[1,1])
    ax.set_xlabel('zk mode')
    ax.set_ylabel('OPD [um]')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
fig,ax = plt.subplots(1,1,figsize=(6,4))
# zkOpd in the header is zk1:zk28. So need to select only from zk4:22
colors = {'intra':'blue', 'extra':'red', 'focal':'green'}
for defocal in opdPerOffset.keys():
    opdPerDet = opdPerOffset[defocal]
    for hduIdx in range(1,10):
        zkOpd = opdPerDet[hduIdx]
        ax.plot(range(4,23), np.array(zkOpd[3:22])/1000., label=f'det {hduIdx}',
               c=colors[defocal])
    ax.set_title('header-based OPD, ')
    #ax.legend(bbox_to_anchor=[1,1])
    ax.set_xlabel('zk mode')
    ax.set_ylabel('OPD [um]')
    ax.set_ylim(-0.2,0.2)

## Note on timing:
    
    

We can check how long each simulation took. The first set used 


     --nodes 2 
    #SBATCH --mem=100G 
    #SBATCH --cpus-per-task=9

and had 

    input.sky_catalog.max_flux: 1e7 

(which translates to effectively not simulatiung 13th mag stars and above; for this visit it was ~3 per CCD looking at GAIA catalog, i.e. <1% of stars per CCD).


The second set used 


    #SBATCH --nodes 4 
    #SBATCH --mem=300G 
    #SBATCH --cpus-per-task=9

and had  no `max_flux` setting, but adding 

    stamp.diffraction_fft: ""

to force no FFT mode even on bright stars. 

Both having 

    output.nproc: 9


They took 


| File      | Defocal | Time [sec] | 
| ----------- | ----------- | ----- | 
| 68659_940.out      | Extra       | 10027.94 | 
| 68659_940.outnoFFT | Extra      | 10555.51 | 
| 68659_941.out      | In-focus        | 9877.72  |
| 68659_941.outnoFFT | In-focus        | 9045.44 | 
| 68659_942.out      |  Intra  |  9581.87 | 
|68659_942.outnoFFT | Intra  | 10936.97 | 

So including these few stars that were brighter than flux of 1e7 nJy did not change the execution time by a lot. 

When it comes to timing, Jim Chiang mentions that increasing eg. `--cpus-per-task: 18` we get only "one processor per CCD, so the other 9 would simply sit idle.   That 1 CCD per processor is a fundamental limitation on how fast things will execute given the current imSim design.  So I don't think any of the tweaks you are proposing would have any positive effect. (--nodes 2 might even run slower since all of the cores used need to be on the same node as the head galsim process).".

So this means that it's best to submit with just 1 node, and match `--cpus-per-task` to `output.nproc` and to `output.nfiles`. Because the fundamental limitation is that we can have only one processor per CCD... 




Josh:  "you might be able to speed things up by, e.g., requesting 9*N processors, still setting output.nproc=9, but putting OMP_NUM_THREADS=N into the env.  batoid should see this and use N threads per process.  The rest of the code should ignore it and proceed like usual.  I suspect for the bright things we’re dominated by batoid so might be an overall win.  But definitely to be viewed as experimental." 

You’d need 9*N for --cpus-per-task presumably



Ok, so this means that the following should speed things up:


    output.nproc=9
    output.nfiles=9

and


    --cpus-per-task: 18 

in the yaml, aas well as 
    
    OMP_NUM_THREADS=2
    
in the env. (I do that in the terminal with  
  
    export OMP_NUM_THREADS=2

and 
  
    echo $OMP_NUM_THREADS
    
to confirm it worked) 
 

In [ ]:
import os
path_cwd = '/sdf/data/rubin/shared/scichris/DM-41679/'
out_files = [file for file in os.listdir(path_cwd) if  '.out' in file]



# find out what is the content and length of all out files 
file_content = {}
file_length = {}

for i in range(len(out_files)):
    fname = out_files[i]
    path_to_file = os.path.join(path_cwd, fname)

    with open(path_to_file) as f:
        allLines = f.readlines()
    file_content[fname] = allLines
    file_length[fname] = len(allLines)
    
    
    
# check only those that seem to be long enough (longer than eg. 700 lines)
# and then check if any  the lines  in the file seems to contain the 
# summary statement that tells us how much time it took ...

# record which ones have actually finished as opposed to not finished ... 
# basically those that have the `Total time` have finished, and everything else has not ... 
file_runtime_sec = {}

for fname  in file_length.keys():
    length = file_length[fname]
    if length>300:
        
        for content in file_content[fname]:
            if content.startswith('Total time for '):

                a=content
                runtime = float(a.split()[-2])
                file_runtime_sec[fname] = runtime
              
       